# Multi-Interval Index Example

This notebook demonstrates how `DimensionIntervalMulti` enables automatic cross-slicing between multiple interval types over a shared continuous dimension.

## Use Case: Speech Data

Imagine you have speech data with:
- A **continuous time dimension** (e.g., audio samples at regular intervals)
- **Word intervals** - each word spans a range of time
- **Phoneme intervals** - each phoneme spans a smaller range of time within words

When you select a specific word, you want the time and phoneme dimensions to automatically constrain to only the overlapping values. This is exactly what `DimensionIntervalMulti` provides.

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

from linked_indices.multi_interval_index import DimensionIntervalMulti

## Creating the Dataset

First, let's create a dataset with:
- 1000 time points from 0 to 120
- 3 words: "run" [0-40), "quickly" [40-80), "home" [80-120)
- 6 phonemes, each spanning 20 time units
- Part of speech labels for each word (verb, adverb, noun)

Note that word intervals are larger and contain multiple phoneme intervals.

In [34]:
C = 2  # number of channels
N = 1000  # number of time points
times = np.linspace(0, 120, N)

# Word intervals - 3 words covering full time range
word_breaks = [0.0, 40.0, 80.0, 120.0]
word_intervals = pd.IntervalIndex.from_breaks(word_breaks, closed="left")
word_labels = ["run", "quickly", "home"]
word_pos = ["verb", "adverb", "noun"]  # part of speech labels

# Phoneme intervals - 6 phonemes, each 20 units
phoneme_breaks = [0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0]
phoneme_intervals = pd.IntervalIndex.from_breaks(phoneme_breaks, closed="left")
phoneme_labels = ["ah", "ee", "oh", "oo", "eh", "ih"]

data = np.random.rand(C, N)

# Create the dataset
ds = xr.Dataset(
    {"data": (("C", "time"), data)},
    coords={
        "time": times,
        "word_intervals": ("word", word_intervals),
        "word": ("word", word_labels),
        "part_of_speech": ("word", word_pos),
        "phoneme_intervals": ("phoneme", phoneme_intervals),
        "phoneme": ("phoneme", phoneme_labels),
    },
)

print("Original Dataset (before applying DimensionIntervalMulti):")
ds

Original Dataset (before applying DimensionIntervalMulti):


<xarray.Dataset> Size: 24kB
Dimensions:            (C: 2, time: 1000, word: 3, phoneme: 6)
Coordinates:
  * time               (time) float64 8kB 0.0 0.1201 0.2402 ... 119.9 120.0
  * word               (word) <U7 84B 'run' 'quickly' 'home'
  * phoneme            (phoneme) <U2 48B 'ah' 'ee' 'oh' 'oo' 'eh' 'ih'
    word_intervals     (word) interval[float64, left] 48B [0.0, 40.0) ... [80...
    part_of_speech     (word) <U6 72B 'verb' 'adverb' 'noun'
    phoneme_intervals  (phoneme) interval[float64, left] 96B [0.0, 20.0) ... ...
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 16kB 0.3927 0.9313 ... 0.01881 0.1274

In [22]:
ds.assign_coords({"hc": ("whatever", [0, 1, 2, 3])})

<xarray.Dataset> Size: 19MB
Dimensions:     (ocean_time: 2, s_rho: 30, eta_rho: 191, xi_rho: 371,
                 whatever: 4)
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 16B 2001-08-01 2001-08-08
  * s_rho       (s_rho) float64 240B -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
    Cs_r        (s_rho) float64 240B -0.933 -0.8092 ... -0.0005206 -5.758e-05
    lon_rho     (eta_rho, xi_rho) float64 567kB -93.6 -93.58 ... -88.88 -88.87
    hc          (whatever) int64 32B 0 1 2 3
    h           (eta_rho, xi_rho) float64 567kB 612.1 610.9 609.7 ... 5.0 5.0
    lat_rho     (eta_rho, xi_rho) float64 567kB 27.45 27.45 ... 30.85 30.86
    Vtransform  int32 4B 2
Dimensions without coordinates: eta_rho, xi_rho, whatever
Data variables:
    salt        (ocean_time, s_rho, eta_rho, xi_rho) float32 17MB 34.95 ... nan
    zeta        (ocean_time, eta_rho, xi_rho) float32 567kB -0.1565 ... nan
Attributes: (12/34)
    file:              ../output_20yr_obc/2001/ocean_his_0015.nc
    format:            netCDF-4/HDF5 file
    Conventions:       CF-1.4
    type:              ROMS/TOMS history file
    title:             TXLA ROMS hindcast run with dyes and oxygen
    rst_file:          ../output_20yr_obc/2001/ocean_rst.nc
    ...                ...
    compiler_flags:    -heap-arrays -fp-model fast -mt_mpi -ip -O3 -msse2 -free
    tiling:            010x012
    history:           Tue Jul 24 11:04:43 2018: /opt/nco/ncks -D 4 -t 8 /cop...
    ana_file:          /home/d.kobashi/TXLA_ROMS_reana/Functionals/ana_btflux...
    CPP_options:       TXLA2, ANA_BPFLUX, ANA_BSFLUX, ANA_BTFLUX, ANA_NUDGCOE...
    NCO:               netCDF Operators version 4.7.6-alpha04 (Homepage = htt...

In [23]:
ds["salt"].values.shape

(2, 30, 191, 371)

## Applying the Multi-Interval Index

Now we apply `DimensionIntervalMulti` to link all these coordinates together. This replaces the default indexes with a single custom index that understands the relationships between time, words, and phonemes.

In [35]:
ds_indexed = ds.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_intervals",
        "phoneme_intervals",
        "word",
        "part_of_speech",
        "phoneme",
    ],
    DimensionIntervalMulti,
)

print("Dataset with DimensionIntervalMulti:")
ds_indexed
ds = ds_indexed

Dataset with DimensionIntervalMulti:


In [38]:
ds

<xarray.Dataset> Size: 24kB
Dimensions:            (C: 2, time: 1000, word: 3, phoneme: 6)
Coordinates:
  * time               (time) float64 8kB 0.0 0.1201 0.2402 ... 119.9 120.0
  * word               (word) <U7 84B 'run' 'quickly' 'home'
  * phoneme            (phoneme) <U2 48B 'ah' 'ee' 'oh' 'oo' 'eh' 'ih'
  * word_intervals     (word) interval[float64, left] 48B [0.0, 40.0) ... [80...
  * phoneme_intervals  (phoneme) interval[float64, left] 96B [0.0, 20.0) ... ...
  * part_of_speech     (word) <U6 72B 'verb' 'adverb' 'noun'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 16kB 0.3927 0.9313 ... 0.01881 0.1274
Indexes:
  ┌ time               DimensionIntervalMulti
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

In [40]:
ds["word_intervals"]

<xarray.DataArray 'word_intervals' (word: 3)> Size: 48B
<IntervalArray>
[[0.0, 40.0), [40.0, 80.0), [80.0, 120.0)]
Length: 3, dtype: interval[float64, left]
Coordinates:
  * word               (word) <U7 84B 'run' 'quickly' 'home'
  * time               (time) float64 8kB 0.0 0.1201 0.2402 ... 119.9 120.0
  * word_intervals     (word) interval[float64, left] 48B [0.0, 40.0) ... [80...
  * phoneme_intervals  (phoneme) interval[float64, left] 96B [0.0, 20.0) ... ...
  * part_of_speech     (word) <U6 72B 'verb' 'adverb' 'noun'
  * phoneme            (phoneme) <U2 48B 'ah' 'ee' 'oh' 'oo' 'eh' 'ih'
Indexes:
  ┌ time               DimensionIntervalMulti
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

In [39]:
ds.sel(part_of_speech="adverb", phoneme="oh")

<xarray.Dataset> Size: 4kB
Dimensions:            (C: 2, time: 167, word: 1, phoneme: 1)
Coordinates:
  * time               (time) float64 1kB 40.0 40.12 40.24 ... 59.7 59.82 59.94
  * word               (word) <U7 28B 'quickly'
  * phoneme            (phoneme) <U2 8B 'oh'
  * word_intervals     (word) interval[float64, left] 16B [40.0, 80.0)
  * phoneme_intervals  (phoneme) interval[float64, left] 16B [40.0, 60.0)
  * part_of_speech     (word) <U6 24B 'adverb'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 3kB 0.7606 0.832 ... 0.4567 0.3576
Indexes:
  ┌ time               DimensionIntervalMulti
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

Notice in the `Indexes` section that all coordinates are now grouped under a single `DimensionIntervalMulti` index. This means selections on any of these coordinates will automatically propagate constraints to the others.

## Cross-Slicing: Selecting on Time

When we select a time range, both word and phoneme dimensions are automatically constrained to only include intervals that overlap with that time range.

In [4]:
# Time 30-70 overlaps:
# - word: [0,40), [40,80) -> 2 words
# - phoneme: [20,40), [40,60), [60,80) -> 3 phonemes
result = ds_indexed.sel(time=slice(30, 70))
_ = result * 1  # force lazy evaluation

print("Selection: time=slice(30, 70)")
print(f"  Time points: {result.sizes['time']}")
print(f"  Words: {result.sizes['word']} (expected: 2)")
print(f"  Phonemes: {result.sizes['phoneme']} (expected: 3)")
print(f"  Word labels: {list(result['word'].values)}")
print(f"  Phoneme labels: {list(result['phoneme'].values)}")

Selection: time=slice(30, 70)
  Time points: 333
  Words: 2 (expected: 2)
  Phonemes: 3 (expected: 3)
  Word labels: [np.str_('run'), np.str_('quickly')]
  Phoneme labels: [np.str_('ee'), np.str_('oh'), np.str_('oo')]


## Cross-Slicing: Selecting on Word Interval

Selecting a point within a word interval selects that word and constrains time and phonemes accordingly.

In [5]:
# Selecting at time=60 picks the word interval [40,80) which is "quickly"
# This constrains:
# - time to 40-80
# - phoneme to [40,60), [60,80) -> 2 phonemes
result = ds_indexed.sel(word_intervals=60)
_ = result * 1

print("Selection: word_intervals=60 (selects word [40,80))")
print(
    f"  Time range: {result['time'].min().values:.1f} to {result['time'].max().values:.1f}"
)
print(f"  Words: {result.sizes['word']}")
print(f"  Phonemes: {result.sizes['phoneme']} (expected: 2)")
print(f"  Word selected: {result['word'].values}")
print(f"  Phonemes overlapping: {list(result['phoneme'].values)}")

Selection: word_intervals=60 (selects word [40,80))
  Time range: 40.0 to 80.0
  Words: 1
  Phonemes: 2 (expected: 2)
  Word selected: ['quickly']
  Phonemes overlapping: [np.str_('oh'), np.str_('oo')]


## Cross-Slicing: Selecting on Phoneme Interval

Similarly, selecting a phoneme constrains time and words.

In [6]:
# Selecting at time=70 picks phoneme interval [60,80)
# This constrains:
# - time to 60-80
# - word to [40,80) -> 1 word ("quickly")
result = ds_indexed.sel(phoneme_intervals=70)
_ = result * 1

print("Selection: phoneme_intervals=70 (selects phoneme [60,80))")
print(
    f"  Time range: {result['time'].min().values:.1f} to {result['time'].max().values:.1f}"
)
print(f"  Words: {result.sizes['word']} (expected: 1)")
print(f"  Phonemes: {result.sizes['phoneme']}")
print(f"  Word overlapping: {result['word'].values}")

Selection: phoneme_intervals=70 (selects phoneme [60,80))
  Time range: 60.1 to 80.0
  Words: 1 (expected: 1)
  Phonemes: 1
  Word overlapping: ['quickly']


## Label-Based Selection

You can also select using the label coordinates (word names, phoneme names, or part of speech). The cross-slicing behavior works the same way.

In [7]:
# Select by word label
result = ds_indexed.sel(word="run")
_ = result * 1

print("Selection: word='run'")
print("  'run' is the first word [0,40)")
print(
    f"  Time range: {result['time'].min().values:.1f} to {result['time'].max().values:.1f}"
)
print(f"  Phonemes: {result.sizes['phoneme']} (expected: 2 - those overlapping [0,40))")
print(f"  Phoneme labels: {list(result['phoneme'].values)}")

Selection: word='run'
  'run' is the first word [0,40)
  Time range: 0.0 to 40.0
  Phonemes: 2 (expected: 2 - those overlapping [0,40))
  Phoneme labels: [np.str_('ah'), np.str_('ee')]


In [8]:
# Select by phoneme label
result = ds_indexed.sel(phoneme="oh")
_ = result * 1

print("Selection: phoneme='oh'")
print("  'oh' is the 3rd phoneme [40,60)")
print(
    f"  Time range: {result['time'].min().values:.1f} to {result['time'].max().values:.1f}"
)
print(f"  Words: {result.sizes['word']} (expected: 1 - 'quickly' which spans [40,80))")
print(f"  Word selected: {result['word'].values}")

Selection: phoneme='oh'
  'oh' is the 3rd phoneme [40,60)
  Time range: 40.0 to 59.9
  Words: 1 (expected: 1 - 'quickly' which spans [40,80))
  Word selected: ['quickly']


## Selection by Part of Speech

The word dimension has two label coordinates: `word` (the word text) and `part_of_speech`. You can select on either one.

In [9]:
# Select by part of speech
result = ds_indexed.sel(part_of_speech="noun")
_ = result * 1

print("Selection: part_of_speech='noun'")
print("  'noun' corresponds to 'home' [80,120)")
print(
    f"  Time range: {result['time'].min().values:.1f} to {result['time'].max().values:.1f}"
)
print(f"  Word selected: {result['word'].values}")
print(
    f"  Phonemes: {result.sizes['phoneme']} (expected: 2 - those overlapping [80,120))"
)
print(f"  Phoneme labels: {list(result['phoneme'].values)}")

Selection: part_of_speech='noun'
  'noun' corresponds to 'home' [80,120)
  Time range: 80.0 to 120.0
  Word selected: ['home']
  Phonemes: 2 (expected: 2 - those overlapping [80,120))
  Phoneme labels: [np.str_('eh'), np.str_('ih')]


## Summary

`DimensionIntervalMulti` provides:

1. **Automatic cross-slicing** - Select on any dimension and all others are constrained appropriately
2. **Multiple interval types** - Support for multiple interval dimensions (words, phonemes, etc.) over a single continuous dimension
3. **Multiple labels per interval** - Each interval dimension can have multiple label coordinates (e.g., word text and part of speech)
4. **Flexible selection** - Use `sel()` with time values, interval coordinates, or label coordinates

This is particularly useful for:
- Speech/audio data with hierarchical annotations
- Time series with multiple granularities of events
- Any data where intervals at different scales need to stay synchronized